### Dart Open Api 사용해서 크롤링

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import webbrowser
import json
import requests

In [2]:
# 개발 참조 https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS003

### 상장된 회사들만 조회하기 위해, 상장회사의 코드를 통해 공시고유번호를 추출한다.

In [3]:
# 상장된 회사들의 회사코드

In [4]:
df_listed = pd.read_csv("상장회사목록.csv", encoding = "ms949")

상장기업 = df_listed["상장코드"]
def delete_A(list_code):
    new_code = list_code[1:]
    return new_code

df_listed["상장코드"] = 상장기업.apply(delete_A)
상장코드 = df_listed["상장코드"]

In [5]:
# 회사공시고유번호정보

In [6]:
df_crpcode = pd.read_csv("회사별 공시고유번호.csv", encoding = "ms949")
df_crpcode = df_crpcode.drop("Unnamed: 0", axis = 1)

In [7]:
# 공시고유번호는 무조건 8자리여야 하기 때문에 8자리 아닌 것들은 앞에 0을 붙여줌
def make_eight_digits(crpcode):
    crpcode = str(crpcode)
    code_length = len(crpcode)
    add_zeros = 8 - code_length
    prefix = add_zeros * str(0)
    return(prefix + crpcode)
    

df_crpcode["공시고유번호"] = df_crpcode["공시고유번호"].apply(make_eight_digits)

#### 상장코드가 존재하는 회사들의 공시고유번호만 뽑아와서 데이터프레임을 만듦

In [8]:
old = pd.DataFrame()
for company in 상장코드:
    new = df_crpcode[df_crpcode["상장코드"] == company]
    old = pd.concat([old, new])

In [9]:
old.head()

,공시고유번호,상장코드,최종수정일,회사명
67511,00126380,005930,20191209,삼성전자
71468,00164779,000660,20191206,SK하이닉스
70948,00877059,207940,20200113,삼성바이오로직스
72717,00266961,035420,20200114,NAVER
75127,00164742,005380,20200110,현대자동차


In [42]:
# 요청값
# 내 API key니까 다들 하나씩 발급받으면 좋을듯!
# 1일 1만회 요청건수 제한
auth_key="847436c641de81093f023d156e8686df4e226471" 
# 사업연도 설정
사업연도 ="2018"
# 1분기보고서 : 11013
# 반기보고서 : 11012
# 3분기보고서 : 11014
# 사업보고서 : 11011
report_code = "11014"

In [43]:
crp_code ="00356370" 

In [44]:
url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key={0}&corp_code={1}&bsns_year={2}&reprt_code={3}"
url = url.format(auth_key, crp_code, 사업연도, report_code)

In [1]:
response = requests.get(url)
data = json.loads(response.content)

data.head()

NameError: name 'requests' is not defined

In [ ]:
# 문제가 2019년 3분기가 다운로드가 안 됨--- 왜 안되는지 모르겠네..